In [2]:
import os
os.getcwd()

'/mnt/c/Users/Komar/Lab/retroparty'

In [3]:
os.chdir('/mnt/c/Users/Komar/Lab')
os.getcwd()

'/mnt/c/Users/Komar/Lab'

In [ ]:
import pandas as pd
flanks = pd.read_table()

In [4]:
import pandas as pd
flank_coords = ["chr3:81813044-81813113",
                "chr2:75940732-75940832",
                "chr1:172593979-172594079"
               ]
res = pd.DataFrame(columns=['flank coord', 'repeat coord', 'strand', 'repeat name', 'link UCSC'])
res

,flank coord,repeat coord,strand,repeat name,link UCSC


In [5]:
%%time
from tqdm import trange
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

service = Service(executable_path="/usr/bin/chromedriver")
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=chrome_options)

# go to the RepeatMasker
driver.get("https://www.repeatmasker.org/cgi-bin/AnnotationRequest")
print(driver.title)

# set Hg38
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "genome"))
    )
    dropdown = Select(element)
    dropdown.select_by_visible_text("Human - Dec 2013 - hg38")
except:
    print("can't select genome")

# flank coords annotating
for i in trange(len(flank_coords)):
    # set flank coords and submit
    try:
        range = driver.find_element(By.NAME, "range")
        range.clear()
        range.send_keys(flank_coords[i])
       
        button = driver.find_element(By.NAME, ".submit")
        button.click()
        error = ''
    except:
        print("can't pass coords")

    # get results 
    try:
        element = WebDriverWait(driver, 60).until(
                  EC.presence_of_element_located((By.TAG_NAME, "pre")) 
                  )
        link = element.find_element(By.TAG_NAME, "a").get_attribute("href")
        list = element.text
        list = list.split('\n')[2].split(' ')
        list = ' '.join(list).split()
        res.loc[len(res.index)] = [flank_coords[i], f"{list[4]}:{list[5]}-{list[6]}", list[8], list[9], link]
    except:
        print('exception 1')
        try:
            #did not find any matching interspersed repeat annotations
            element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, "/html/body/form/font/text()")) 
            )
            res.loc[len(res.index)] = [flank_coords[i], "-", "strand", "-", "-"]
        except:
            print("exception 2")

    # get back to request form
    driver.back()


#quit from site
driver.quit()
res

Repeat Annotation Request Form


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:19<00:00,  6.52s/it]

CPU times: user 124 ms, sys: 0 ns, total: 124 ms
Wall time: 22.5 s


,flank coord,repeat coord,strand,repeat name,link UCSC
0,chr3:81813044-81813113,chr3:81813104-81813389,+,AluYa5,http://genome.ucsc.edu/cgi-bin/hgTracks?db=hg3...
1,chr2:75940732-75940832,chr2:75940534-75940823,+,AluYk2,http://genome.ucsc.edu/cgi-bin/hgTracks?db=hg3...
2,chr1:172593979-172594079,chr1:172591275-172597414,C,L1PA5,http://genome.ucsc.edu/cgi-bin/hgTracks?db=hg3...


In [11]:
flank_fa = ">1_R1\nGACTCTTCTTACTTTTAAGGACACATATAATTGAAACTATTTTGGAAAACAAGAGAGCAGGGATAG\n>2_R1\nTACTATAAACACTTCTATGCCAATAAACTAGAAAATCCAGAAGAAATGGATAAATTCCTGGACACATACACCCTCCCAAGAC"

In [26]:
%%time
import pandas as pd
blat_out = pd.DataFrame(columns=['QUERY', 'SCORE', 'START', 'END', 'QSIZE', 'IDENTITY', 'CHROM', 'STRAND', 'START', 'END', 'SPAN'])

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

service = Service(executable_path="/usr/bin/chromedriver")
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=chrome_options)

# go to the UCSC Blat
driver.get("https://genome.ucsc.edu/cgi-bin/hgBlat?hgsid=1711647570_KRW9QfFHAHaje8cp8sWT4EtpGupE&command=start")
print(driver.title)

# set Hg38
try:
    selects = driver.find_elements(By.TAG_NAME, "select")
    for i in selects:
        if i.get_property("name") == "db":
            element = i
            break
    dropdown = Select(element)
    dropdown.select_by_visible_text("Dec. 2013 (GRCh38/hg38)")
except:
    print("can't select genome")
    

# set flank coords and submit
try:
    range = driver.find_element(By.NAME, "userSeq")
    range.clear()
    range.send_keys(flank_fa)
   
    button = driver.find_element(By.NAME, "Submit")
    button.click()
except:
    print("can't pass coords")
    driver.quit()
    
# get results 
try:
    element = WebDriverWait(driver, 60).until(
                  EC.presence_of_element_located((By.TAG_NAME, "pre")) 
                  )
    list = element.text
    list = list.split('\n')
    for line in list:
        if line.count('browser'):
            tmp = line.split(' ')
            tmp = ' '.join(tmp).split()
            blat_out.loc[len(blat_out.index)] = [tmp[2], tmp[3], tmp[4], tmp[5], tmp[6], tmp[7], tmp[8], tmp[9], tmp[10], tmp[11], tmp[12]]
except:
    print('no results')

# get back to request form
driver.quit()
blat_out

Human BLAT Search
CPU times: user 286 ms, sys: 2.64 ms, total: 289 ms
Wall time: 5.94 s


,QUERY,SCORE,START,END,QSIZE,IDENTITY,CHROM,STRAND,START,END,SPAN
0,1_R1,66,1,66,66,100.0%,chr1_KV880763v1_alt,-,290696,290761,66
1,1_R1,66,1,66,66,100.0%,chr1,-,108278702,108278767,66
2,1_R1,66,1,66,66,100.0%,chr1_KV880763v1_alt,+,427459,427524,66
3,1_R1,66,1,66,66,100.0%,chr1,+,108415614,108415679,66
4,1_R1,25,25,52,66,96.5%,chr3,+,30191242,30191272,31
...,...,...,...,...,...,...,...,...,...,...,...
437,2_R1,68,1,82,82,91.5%,chr13,+,79978580,79978661,82
438,2_R1,68,1,82,82,91.5%,chr1,+,187141065,187141146,82
439,2_R1,67,2,82,82,91.4%,chr5,-,159686811,159686891,81
440,2_R1,66,1,82,82,90.3%,chr10,-,23592506,23592587,82
